In [1]:
%matplotlib inline

In [11]:
import cv2
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from tensorflow.keras.optimizers import Adam,SGD
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from mostCommon import most_common,patient_score

In [12]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224

In [13]:
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})


In [14]:
testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]


In [15]:
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0

In [16]:
#Applying SMOTE
oversample = SMOTE()
X_train=np.resize(X_train,(X_train.shape[0],50176))
X_train,y_train=oversample.fit_resample(X_train, y_train)
counter=Counter(y_train)
print(counter)

Counter({'covid': 225, 'pneumonia': 225, 'regular': 225})


In [17]:
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))

X_train: (675, 50176)
y_train: (675,)
X_test: (120, 224, 224, 1)
y_test: (120,)


In [18]:
model=DecisionTreeClassifier(random_state=42,criterion="entropy", max_depth=1000)
model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=1000, random_state=42)

In [19]:
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)

In [20]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)

[[23  1  6]
 [ 1 28  6]
 [ 0  5 50]]
              precision    recall  f1-score   support

       covid      0.958     0.767     0.852        30
   pneumonia      0.824     0.800     0.812        35
     regular      0.806     0.909     0.855        55

    accuracy                          0.842       120
   macro avg      0.863     0.825     0.839       120
weighted avg      0.849     0.842     0.841       120

[[ 5  0  1]
 [ 0  5  2]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.833     0.909         6
   pneumonia      0.833     0.714     0.769         7
     regular      0.769     0.909     0.833        11

    accuracy                          0.833        24
   macro avg      0.868     0.819     0.837        24
weighted avg      0.846     0.833     0.834        24



In [21]:
>>> from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.83 accuracy with a standard deviation of 0.11


In [22]:


IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0
X_train=np.resize(X_train,(X_train.shape[0],50176))
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=DecisionTreeClassifier(random_state=42,criterion="entropy", max_depth=1000)
model.fit(X_train,y_train)
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})
X_train: (490, 50176)
y_train: (490,)
X_test: (120, 224, 224, 1)
y_test: (120,)
[[17  2 11]
 [ 0 22 13]
 [ 1  3 51]]
              precision    recall  f1-score   support

       covid      0.944     0.567     0.708        30
   pneumonia      0.815     0.629     0.710        35
     regular      0.680     0.927     0.785        55

    accuracy                          0.750       120
   macro avg      0.813     0.708     0.734       120
weighted avg      0.785     0.750     0.744       120

[[ 3  0  3]
 [ 0  5  2]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.500     0.667         6
   pneumonia      0.833     0.714     0.769         7
     regular      0.667     0.909     0.769        11

    accuracy                          0.750        24
   macro avg      0.833     0.708     0.735        24
weighted avg      0.799     0.750     0.744        24

0.65 accuracy with a stand

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set

X = scaler.transform(X)
pca = PCA(.95)
pca.fit(X)
X = pca.transform(X)

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)


print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=DecisionTreeClassifier(random_state=42,criterion="entropy", max_depth=1000)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})
X_train: (490, 99)
y_train: (490,)
X_test: (120, 99)
y_test: (120,)
[[15  1 14]
 [ 6 19 10]
 [ 4  8 43]]
              precision    recall  f1-score   support

       covid      0.600     0.500     0.545        30
   pneumonia      0.679     0.543     0.603        35
     regular      0.642     0.782     0.705        55

    accuracy                          0.642       120
   macro avg      0.640     0.608     0.618       120
weighted avg      0.642     0.642     0.635       120

[[3 0 3]
 [1 5 1]
 [1 1 9]]
              precision    recall  f1-score   support

       covid      0.600     0.500     0.545         6
   pneumonia      0.833     0.714     0.769         7
     regular      0.692     0.818     0.750        11

    accuracy                          0.708        24
   macro avg      0.709     0.677     0.688        24
weighted avg      0.710     0.708     0.704        24

0.56 accuracy with a standard deviation of 0.08